# Functions du programme Open Loop sans FeedForward

## 1. Import

In [65]:
from signal import signal
from tokenize import Single
import numpy as np
from scipy.optimize import minimize
from matplotlib import colors as mcolors

%matplotlib qt
import matplotlib.pyplot as plt

from matplotlib.widgets import Slider, Button, RadioButtons,TextBox,CheckButtons
import matplotlib.patches as mpatches
import datetime
import os
import tclab
import time

from xmlrpc.client import Boolean
import numpy as np

from IPython.display import display, clear_output

import package_Class
from package_Class import Simulation,Path,FirstOrder,SecondOrderPlusDelay,LeadLag,FeedForward,PID_Controller,Delay,Signal,Variable,Graph,LabValues

## 2. Création Instance Classes


In [66]:
SIM = Simulation(2000,1,20,True,'Exp_CLP_FF_IMC')

G = Graph(SIM,'PID Control_')

SP = Path(SIM,{0:40 , 1000:60 , SIM.TSim: 60})
DV = Path(SIM,{0:30 , 500: 30 , 1500:50 ,SIM.TSim: 50})
MAN = Path(SIM,{0:1 ,250: 0, SIM.TSim: 0})
MANV = Path(SIM,{0: 50, SIM.TSim: 30})

P = FirstOrder(SIM,0.6576982674557728,201.32536449625266,1,50,SIM.PVInit)
D = FirstOrder(SIM,0.4022800628236135,151.76786567122514,28.926501641455957,50,0)

P_delay = Delay(SIM,P.Theta)
D_delay = Delay(SIM,D.Theta)


FF = FeedForward(SIM,P,D,True)

PID = PID_Controller(SIM,5,10,0,0.14,[0,100],False,False)
PID.IMC_tuning(P,0.5,'G')

 [elapsed time: 0:00:00] |                                  | (ETA:  --:--:--) 

## 3. Boucle pour une simulation / Experience
Si la variable SIM.sim de la simulation est vrais.
Class.RT calcule les valeurs du bloc en instantanée

In [67]:
if(SIM.sim == True):
    t = []
    for ti in SIM.t:
        t.append(ti)
        #Signals
        SP.RT(t)
        DV.RT(t)
        MAN.RT(t)
        MANV.RT(t)

        FF.RT(DV.Signal) # FeedForward
        PID.RT(SP.Signal,SIM.PV,MAN.Signal,MANV.Signal,FF.MVFF,'EBD-EBD')

        SIM.addMV(PID.MVFB,FF.MVFF) # Modified Value

        P_delay.RT(SIM.MV)
        D_delay.RT(DV.Signal)
        
        P.RT(P_delay.PV,'EBD')
        D.RT(D_delay.PV,'EBD')
        
        SIM.PV.append(P.PV[-1]+D.PV[-1]) # Point Value
        SIM.updateBar()


Si la variable SIM.sim de la simulation est fausse alors ouvre la connection TcLab 

In [68]:
if(SIM.sim == False):
    #Tc Lab
    LAB = tclab.TCLab()
    LABVal = LabValues(SIM,LAB)

    SIM.t = []
    start = time.time()
    delta = 0
    totalTime = 0
    last = time.time()

    while totalTime < SIM.TSim:
        if delta > SIM.Ts:
            last = time.time()
            SIM.t.append(round(totalTime,4))
            #Signals
            SP.RT(SIM.t)
            DV.RT(SIM.t)
            MAN.RT(SIM.t)
            MANV.RT(SIM.t)

            FF.RT(DV.Signal) # FeedForward
            PID.RT(SP.Signal,SIM.PV,MAN.Signal,MANV.Signal,FF.MVFF,'EBD-EBD')

            SIM.addMV(PID.MVFB,FF.MVFF) # Modified Value

            LABVal.RT(SIM.MV,DV.Signal,D.point_fct)
            delta = 0
            SIM.updateBar()

        else :
            totalTime = time.time() - start
            delta = time.time() - last
    LAB.close()
        

TCLab version 0.4.9
Arduino Leonardo connected on port COM4 at 115200 baud.
TCLab Firmware 1.4.3 Arduino Leonardo/Micro.


 [elapsed time: 0:33:24] |█████████████████████████████████ | (ETA:   0:00:01) 

TCLab disconnected successfully.


## 4. On cree des liste d'instances Signal que l'on veux afficher 
Il suffit de mettre en commentaire ou rajouter pour les afficher

In [71]:
# Signaux Afficher dans  Graph Binaires
SigValsBin = [
    Signal(MAN.Signal,'Manual Mode','-g')
]

# Signaux Afficher dans  Graph 1 : Temperature
SigVals1 = [
    Signal(SP.Signal,'SP','-r'),
    Signal(SIM.PV,'PV','-b'),
    Signal(P.PV,'P(s)','--b'),
    Signal(D.PV,'D(s)','--k'),

]

# Signaux Afficher dans Graph 2 : % de chauffe
SigVals2 = [
    
    Signal(SIM.MV,'MV','-b'),
    Signal(DV.Signal,'DV','-k'),
    #Signal(MANV.Signal,'MANVal','-m'),
    Signal(FF.MVFF,'MVFF','-g'),
    Signal(PID.MVFB,'MVFB','-y'),
    #Signal(PID.E,'E',':r'),
    #Signal(PID.MVP,'MVP',':g'),
    #Signal(PID.MVI,'MVI',':y'),
    #Signal(PID.MVD,'MVD',':m'),
    #Signal(DV.Signal,'DV','-k'),
]

# Signaux enregistrer dans le .txt
SigSave = [
    Signal(SIM.MV,'MV','-b'),
    Signal(PID.MVP,'MVP',':b'),
    Signal(PID.MVI,'MVI',':y'),
    Signal(PID.MVD,'MVD',':m'),
    Signal(SP.Signal,'SP',':m'),
    Signal(SIM.PV,'PV',':m'),
    Signal(DV.Signal,'DV','-k'),
    Signal(MAN.Signal,'Man','-k'),
]

# Variables affichées sur le graph
varVals = [
    Variable(SIM.TSim,'Temps Sim [s]'),
    Variable(SIM.Ts,'Sampling [s]'),
    Variable(SIM.PVInit,'Pv Init [°C]'),

    Variable(PID.OLP,'Open Loop'),
    Variable(PID.ManFF,'Man FF'),


    Variable(PID.Kc,'Kc PID'),
    Variable(PID.Td,'Td PID'),
    Variable(PID.Ti,'Ti PID'),
    Variable(PID.gamma,'Gamma IMC'),

    Variable(FF.active,'FF Enabled'),
    Variable(FF.T1p,'TLead P(s)'),
    Variable(FF.T2p,'TLag P(s)'),
    Variable(FF.T1d,'TLead D(s)'),
    Variable(FF.T2d,'TLag D(s)'),

]

## 5. Affichage du graph des signaux et bode

Marche pas bien dans kernel -> MathPlotLib Ram (Fermer Alt F4 pas bouton close)

In [72]:
G.show([SigVals1,SigVals2,SigSave],SigValsBin,varVals)

ValueError: x and y must have same first dimension, but have shapes (1998,) and (0,)

In [ ]:
G.Bode(P,PID,'PID')

In [ ]:
#G.Bode(P,PID,'P')